# Amazon Bedrock AgentCore Gateway ツールレスポンスでの機密データのマスキング

## 概要

このノートブックでは、**Amazon Bedrock Guardrails** と統合した **Amazon Bedrock AgentCore Gateway インターセプター**を使用して、ツールレスポンス内の**個人識別情報（PII）を自動的に匿名化**する方法を示します。インターセプターはツールレスポンスをリアルタイムで検査し、Bedrock の組み込み PII 検出と匿名化機能を使用して機密データを匿名化してからクライアントに結果を返し、データプライバシー規制へのコンプライアンスを確保します。

### なぜ Gateway で機密データをマスクするのか？

顧客データにアクセスする AI アプリケーションを構築する際には、機密情報を保護する必要があります：

- **コンプライアンス**: GDPR、HIPAA、PCI-DSS、その他の規制要件を満たす
- **データ最小化**: 必要な情報のみをクライアントに公開
- **一元化された保護**: すべてのツールに一貫して匿名化ルールを適用
- **ゼロトラスト**: 下流システムが機密データを保護することに依存しない
- **AI 駆動の検出**: 31 以上の PII タイプにわたる Bedrock Guardrails の高度な PII 検出を活用

Gateway インターセプターは、どのツールがそれを返すかに関係なく PII を匿名化する**一元化された実施ポイント**を提供し、個々のツール実装を変更する必要はありません。

---

## このチュートリアルでカバーする内容

このチュートリアルでは、**Amazon Bedrock Guardrails** を使用した**レスポンスインターセプター**で PII 匿名化を実装します：

🔒 **PII 匿名化（レスポンスインターセプター + Bedrock Guardrails）**
   - 包括的な検出のために 31 以上の PII タイプで構成された Bedrock Guardrail を作成
   - Gateway からのツールレスポンスをインターセプト
   - Bedrock Guardrails を適用して機密データ（メール、電話番号、SSN、クレジットカード、住所など）を検出し匿名化
   - 検出された PII を匿名化されたプレースホルダー（例：`[EMAIL]`、`[PHONE]`）に置換
   - サニタイズされたレスポンスをクライアントに返す

![PII Masking Architecture](images/PII-mask.png)

---

## なぜ Gateway インターセプターを使用するのか？

Gateway インターセプターにより、以下のことが可能になります：

- **データ保護**: AI 駆動の検出を使用してレスポンスから機密情報を自動的に匿名化
- **コンプライアンス実施**: すべてのツールに一貫したデータ保護ポリシーを適用
- **包括的なカバレッジ**: 名前、住所、財務データ、健康情報など 31 以上の PII タイプを検出
- **監査とガバナンス**: データアクセスと匿名化イベントをログに記録
- **レスポンス変換**: ツールを変更せずに転送中のデータを変更
- **マネージドサービス**: Bedrock Guardrails の継続的に更新される PII 検出モデルを活用

インターセプターは **Gateway レイヤー**にアタッチされているため、アプリケーションコードを変更せずに**任意の**基盤となる MCP サーバーまたはツールからのデータを保護します。

---

## チュートリアルの詳細

| 情報                     | 詳細                                                                         |
|--------------------------|------------------------------------------------------------------------------|
| **チュートリアルタイプ** | インタラクティブ                                                             |
| **AgentCore コンポーネント** | Amazon Bedrock AgentCore Gateway、Gateway インターセプター、Bedrock Guardrails |
| **Gateway ターゲットタイプ** | MCP サーバー（Lambda ベースのツール）                                       |
| **インターセプタータイプ** | AWS Lambda（レスポンス）                                                    |
| **インバウンド認証 IdP** | Amazon Cognito（CUSTOM_JWT オーソライザー）                                  |
| **データ保護**           | Amazon Bedrock Guardrails を使用した PII 匿名化（31 以上の PII タイプ）      |
| **チュートリアルコンポーネント** | Gateway、Lambda インターセプター、Bedrock Guardrails、Amazon Cognito、MCP ツール |
| **チュートリアル分野**   | クロスバーティカル（PII を扱うあらゆる業界に適用可能）                       |
| **難易度**               | 中級                                                                         |
| **使用 SDK**             | boto3                                                                        |

---

## 前提条件

このチュートリアルを実行するには、以下が必要です：

- Jupyter notebook（Python カーネル）
- 以下の権限を持つ AWS 資格情報：
  - AWS Lambda
  - AWS IAM
  - Amazon Cognito
  - Amazon Bedrock AgentCore サービス（コントロールプレーン）
  - Amazon Bedrock Guardrails（bedrock:CreateGuardrail、bedrock:ApplyGuardrail）
- Python 3.9 以上
- AWS Lambda、IAM ロール、Amazon Cognito、Amazon Bedrock Guardrails、Amazon Bedrock AgentCore Gateway の基本的な理解

> ⚠️ **注意：** 最後のクリーンアップセクションは、このチュートリアルで作成された AWS リソース（Gateway、Lambda、IAM ロール、Guardrails など）を削除します。すべてを削除する準備ができたときにのみ実行してください。

---

## パート 1: セットアップとデプロイ

### ステップ 1.0: 必要な依存関係のインストール

このチュートリアルに必要なすべての Python パッケージをインストールします。

In [ ]:
!pip install -r requirements.txt

### ステップ 1.1: 必要なライブラリのインポート

In [ ]:
import boto3
import json
import time
import sys
from pathlib import Path
from datetime import datetime
from botocore.exceptions import ClientError

# Add parent directory to path for utils
utils_dir = Path.cwd().parent
sys.path.insert(0, str(utils_dir))

import utils

print("✓ Libraries imported")

# Generate unique identifier for this deployment
DEPLOYMENT_ID = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"\nDeployment ID: {DEPLOYMENT_ID}")

### ステップ 1.2: デプロイ変数の設定

In [ ]:
# Configuration
REGION = "us-east-1"  
LAMBDA_FUNCTION_NAME = f"interceptor-lambda-{DEPLOYMENT_ID}"
LAMBDA_ROLE_NAME = f"interceptor-lambda-role-{DEPLOYMENT_ID}"
GATEWAY_NAME = f"interceptor-gateway-{DEPLOYMENT_ID}"

# Initialize clients
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
cognito_client = boto3.client('cognito-idp', region_name=REGION)

print("Configuration:")
print(f"  Lambda Function: {LAMBDA_FUNCTION_NAME}")
print(f"  Lambda Role: {LAMBDA_ROLE_NAME}")
print(f"  Gateway Name: {GATEWAY_NAME}")
print(f"  Region: {REGION}")


### ステップ 1.3: 機密データフィルタリング用の Bedrock Guardrails のセットアップ

Amazon Bedrock の組み込み機能を使用して PII を匿名化するための、機密データフィルター付き Bedrock Guardrail を作成します。

In [ ]:
# Setup Bedrock Guardrails for sensitive data filtering
print("Creating Bedrock Guardrail with sensitive data filters...")

bedrock_client = boto3.client('bedrock', region_name=REGION)

GUARDRAIL_NAME = f"pii-masking-guardrail-{DEPLOYMENT_ID}"

# Define sensitive data filters with ANONYMIZE behavior
# Bedrock Guardrails supports 31 PII entity types by default
# ANONYMIZE action replaces PII with placeholder text (masking behavior)
sensitive_information_policy_config = {
    'piiEntitiesConfig': [
        # General PII
        {'type': 'ADDRESS', 'action': 'ANONYMIZE'},
        {'type': 'AGE', 'action': 'ANONYMIZE'},
        {'type': 'EMAIL', 'action': 'ANONYMIZE'},
        {'type': 'NAME', 'action': 'ANONYMIZE'},
        {'type': 'PHONE', 'action': 'ANONYMIZE'},
        {'type': 'USERNAME', 'action': 'ANONYMIZE'},
        {'type': 'PASSWORD', 'action': 'ANONYMIZE'},
        
        # Financial Information
        {'type': 'CREDIT_DEBIT_CARD_CVV', 'action': 'ANONYMIZE'},
        {'type': 'CREDIT_DEBIT_CARD_EXPIRY', 'action': 'ANONYMIZE'},
        {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'PIN', 'action': 'ANONYMIZE'},
        {'type': 'INTERNATIONAL_BANK_ACCOUNT_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'SWIFT_CODE', 'action': 'ANONYMIZE'},
        
        # US-Specific Identifiers
        {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'US_BANK_ROUTING_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'US_INDIVIDUAL_TAX_IDENTIFICATION_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'US_PASSPORT_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'DRIVER_ID', 'action': 'ANONYMIZE'},
        
        # UK-Specific Identifiers
        {'type': 'UK_NATIONAL_HEALTH_SERVICE_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'UK_NATIONAL_INSURANCE_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'UK_UNIQUE_TAXPAYER_REFERENCE_NUMBER', 'action': 'ANONYMIZE'},
        
        # Canada-Specific Identifiers
        {'type': 'CA_HEALTH_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'CA_SOCIAL_INSURANCE_NUMBER', 'action': 'ANONYMIZE'},
        
        # Network & Technical
        {'type': 'IP_ADDRESS', 'action': 'ANONYMIZE'},
        {'type': 'MAC_ADDRESS', 'action': 'ANONYMIZE'},
        {'type': 'URL', 'action': 'ANONYMIZE'},
        
        # AWS Credentials
        {'type': 'AWS_ACCESS_KEY', 'action': 'ANONYMIZE'},
        {'type': 'AWS_SECRET_KEY', 'action': 'ANONYMIZE'},
        
        # Vehicle Identification
        {'type': 'VEHICLE_IDENTIFICATION_NUMBER', 'action': 'ANONYMIZE'},
        {'type': 'LICENSE_PLATE', 'action': 'ANONYMIZE'}
    ]
}

try:
    # Create the guardrail
    guardrail_response = bedrock_client.create_guardrail(
        name=GUARDRAIL_NAME,
        description='Guardrail for anonymizing sensitive PII data in tool responses',
        sensitiveInformationPolicyConfig=sensitive_information_policy_config,
        blockedInputMessaging='Input contains sensitive information that has been anonymized.',
        blockedOutputsMessaging='Output contains sensitive information that has been anonymized.'
    )
    
    GUARDRAIL_ID = guardrail_response['guardrailId']
    GUARDRAIL_ARN = guardrail_response['guardrailArn']
    
    print(f"✓ Guardrail created: {GUARDRAIL_ID}")
    print(f"  ARN: {GUARDRAIL_ARN}")
    
    # Create a version of the guardrail
    print("\nCreating guardrail version...")
    version_response = bedrock_client.create_guardrail_version(
        guardrailIdentifier=GUARDRAIL_ID,
        description='Initial version with PII anonymization'
    )
    
    GUARDRAIL_VERSION = version_response['version']
    print(f"✓ Guardrail version created: {GUARDRAIL_VERSION}")
    
    # Display configured PII types
    print(f"\n✓ Configured {len(sensitive_information_policy_config['piiEntitiesConfig'])} PII types for anonymization:")
    for pii_config in sensitive_information_policy_config['piiEntitiesConfig']:
        print(f"  - {pii_config['type']}: {pii_config['action']}")
    
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == 'ConflictException':
        print(f"⚠ Guardrail with name '{GUARDRAIL_NAME}' already exists")
        # List existing guardrails to get the ID
        list_response = bedrock_client.list_guardrails()
        for guardrail in list_response.get('guardrails', []):
            if guardrail['name'] == GUARDRAIL_NAME:
                GUARDRAIL_ID = guardrail['id']
                GUARDRAIL_ARN = guardrail['arn']
                print(f"  Using existing guardrail: {GUARDRAIL_ID}")
                
                # Get the latest version of the existing guardrail
                try:
                    get_response = bedrock_client.get_guardrail(
                        guardrailIdentifier=GUARDRAIL_ID
                    )
                    GUARDRAIL_VERSION = get_response.get('version', 'DRAFT')
                    print(f"  Guardrail version: {GUARDRAIL_VERSION}")
                except Exception as get_error:
                    print(f"  ⚠ Could not get guardrail version: {get_error}")
                    GUARDRAIL_VERSION = 'DRAFT'
                break
    else:
        print(f"✗ Failed to create guardrail: {e}")
        raise
except Exception as e:
    print(f"✗ Unexpected error creating guardrail: {e}")
    raise


### ステップ 1.4: Lambda インターセプター用の IAM ロールの作成

Lambda に実行と CloudWatch ログ書き込みの権限を付与します。

In [ ]:
# Create IAM role for Lambda interceptor using utils
print("Creating IAM role for Lambda interceptor...")

LAMBDA_ROLE_ARN = utils.create_lambda_role(
    role_name=LAMBDA_ROLE_NAME,
    description='Role for AgentCore Lambda Interceptor for PII masking with Bedrock Guardrails'
)

print(f"  ARN: {LAMBDA_ROLE_ARN}")

# Add Bedrock Guardrails permissions to Lambda role
print("\nAdding Bedrock Guardrails permissions to Lambda role...")
iam_client = boto3.client('iam')

bedrock_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:ApplyGuardrail"
            ],
            "Resource": "*"
        }
    ]
}

try:
    iam_client.put_role_policy(
        RoleName=LAMBDA_ROLE_NAME,
        PolicyName='BedrockGuardrailsPolicy',
        PolicyDocument=json.dumps(bedrock_policy)
    )
    print("✓ Bedrock Guardrails permissions added")
    print("  Policy: bedrock:ApplyGuardrail on all resources")
except Exception as e:
    print(f"⚠ Failed to add Bedrock permissions: {e}")

### ステップ 1.4a: Bedrock Guardrail の準備完了を待機

Bedrock Guardrail が伝播し、完全に利用可能になるまで待機します。

In [ ]:
time.sleep(10)

### ステップ 1.5: Lambda インターセプター関数のデプロイ

Lambda はツールレスポンスをインターセプトし、Bedrock Guardrails を使用して PII をマスクします。

In [ ]:
# Deploy Lambda interceptor using utils
print("Deploying Lambda interceptor...")

# Prepare environment variables for Lambda
lambda_env_vars = {}
if 'GUARDRAIL_ID' in globals():
    lambda_env_vars['GUARDRAIL_ID'] = GUARDRAIL_ID
    lambda_env_vars['GUARDRAIL_VERSION'] = GUARDRAIL_VERSION
    print(f"  Configuring Lambda with Guardrail: {GUARDRAIL_ID} (version: {GUARDRAIL_VERSION})")
else:
    print("  ⚠ WARNING: GUARDRAIL_ID not found. Lambda will skip PII masking.")
    print("  Make sure to run Step 1.2a to create the Guardrail first.")

LAMBDA_ARN = utils.deploy_lambda_function(
    function_name=LAMBDA_FUNCTION_NAME,
    role_arn=LAMBDA_ROLE_ARN,
    lambda_code_path='src/lambda/lambda_function.py',
    description='AgentCore Response Lambda Interceptor to mask sensitive data using Bedrock Guardrails',
    timeout=30,
    memory_size=256,
    environment_vars=lambda_env_vars if lambda_env_vars else None,
    region=REGION
)

print(f"  ARN: {LAMBDA_ARN}")

### ステップ 1.5a: Gateway に Lambda 呼び出し権限を付与

Gateway が Lambda インターセプター関数を呼び出すための権限を追加します。

In [ ]:
# Grant Gateway permission to invoke the Lambda interceptor
print("\nGranting Gateway permission to invoke Lambda...")

utils.grant_gateway_invoke_permission(
    function_name=LAMBDA_FUNCTION_NAME,
    region=REGION
)

### ステップ 1.6: Amazon Cognito ユーザープールとアプリクライアントの作成

OAuth クライアント資格情報フローを使用した Gateway 認証用の Cognito ユーザープールを作成します。

In [ ]:
# Create Cognito User Pool and Client for Gateway authentication using utils
print("Creating Cognito User Pool and Client...")

USER_POOL_NAME = f"gateway-pool-{DEPLOYMENT_ID}"
RESOURCE_SERVER_ID = 'gateway'
RESOURCE_SERVER_NAME = 'Gateway Resource Server'
SCOPES = [{'ScopeName': 'tools', 'ScopeDescription': 'Access to gateway tools'}]

# Create or get user pool
USER_POOL_ID = utils.get_or_create_user_pool(cognito_client, USER_POOL_NAME)
print(f"  Pool ID: {USER_POOL_ID}")

# Create or get resource server
utils.get_or_create_resource_server(cognito_client, USER_POOL_ID, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)

# Wait for resource server to propagate
print("  Waiting for resource server to propagate...")
time.sleep(3)

# Create M2M client with client credentials flow
CLIENT_NAME = f"gateway-client-{DEPLOYMENT_ID}"
CLIENT_ID, CLIENT_SECRET = utils.get_or_create_m2m_client(
    cognito_client,
    USER_POOL_ID,
    CLIENT_NAME,
    RESOURCE_SERVER_ID,
    SCOPES=[f"{RESOURCE_SERVER_ID}/tools"]
)

print(f"✓ User Pool Client created: {CLIENT_NAME}")
print(f"  Client ID: {CLIENT_ID}")
print(f"  Client Secret: {CLIENT_SECRET[:20]}...")

# Construct OAuth URLs
POOL_DOMAIN = USER_POOL_ID.replace('_', '').lower()
COGNITO_DOMAIN = f"https://{POOL_DOMAIN}.auth.{REGION}.amazoncognito.com"
DISCOVERY_URL = f"https://cognito-idp.{REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"
TOKEN_URL = f"{COGNITO_DOMAIN}/oauth2/token"

print(f"\n✓ OAuth Configuration:")
print(f"  Discovery URL: {DISCOVERY_URL}")
print(f"  Token URL: {TOKEN_URL}")
print(f"  Scope: {RESOURCE_SERVER_ID}/tools")

### ステップ 1.7: レスポンスインターセプター付き Gateway の作成

**なぜレスポンスインターセプターなのか？**
インターセプターはツール実行後にツールレスポンスを処理し、クライアントにデータを返す前に PII をマスクできるようにします。

In [ ]:
# Create Gateway IAM role
gateway_iam_role = utils.create_agentcore_gateway_role_with_region(GATEWAY_NAME, REGION)
GATEWAY_ROLE_ARN = gateway_iam_role['Role']['Arn']

print(f"✓ Gateway role created: {GATEWAY_ROLE_ARN}")

# Wait for role propagation
time.sleep(10)

# Create Gateway with Lambda interceptor
print(f"\nCreating Gateway with RESPONSE interceptor...")

try:
    gateway_response = gateway_client.create_gateway(
        name=GATEWAY_NAME,
        protocolType="MCP",
        protocolConfiguration={
            "mcp": {
                "supportedVersions": ["2025-03-26"]
            }
        },
        interceptorConfigurations=[
            {
                "interceptor": {
                    "lambda": {
                        "arn": LAMBDA_ARN
                    }
                },
                "interceptionPoints": ["RESPONSE"],
                "inputConfiguration": {
                    "passRequestHeaders": True  
                }
            }
        ],
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": DISCOVERY_URL,
                "allowedClients": [CLIENT_ID]
            }
        },
        roleArn=GATEWAY_ROLE_ARN
    )
    
    GATEWAY_ID = gateway_response.get('gatewayId')
    print(f"✓ Gateway created: {GATEWAY_ID}")
    
except Exception as e:
    print(f"\n✗ Failed to create Gateway: {e}")
    raise


### ステップ 1.8: Gateway の準備完了を待機

In [ ]:
# Wait for Gateway to be ready using signed requests
print("\nWaiting for Gateway to be ready...")

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = gateway_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
        status_code = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

        if status_code == 200:
            # gateway_info = response.json()
            status = response.get('status', 'UNKNOWN')
            
            print(f"  [{attempt + 1}/{max_attempts}] Status: {status}")
            
            if status == 'READY':
                GATEWAY_URL = response.get('gatewayUrl')
                print(f"\n✓ Gateway is ready!")
                print(f"  URL: {GATEWAY_URL}")
                
                # Show interceptor configuration
                if 'interceptorConfigurations' in response:
                    interceptor_configs = response['interceptorConfigurations']
                    print(f"\n  Interceptor Configuration:")
                    for idx, config in enumerate(interceptor_configs):
                        print(f"    [{idx}] Interception Points: {config.get('interceptionPoints', [])}")
                        print(f"    [{idx}] Lambda ARN: {config.get('interceptor', {}).get('lambda', {}).get('arn', 'N/A')}")
                        print(f"    [{idx}] Pass Headers: {config.get('inputConfiguration', {}).get('passRequestHeaders', False)}")
                break
            elif status == 'FAILED':
                print(f"\n✗ Gateway creation failed")
                print(f"  Details: {response}")
                raise Exception("Gateway failed")
        else:
            print(f"  [{attempt + 1}/{max_attempts}] HTTP Error: {response.status_code}")
    except Exception as e:
        print(f"  [{attempt + 1}/{max_attempts}] Error: {e}")
    
    time.sleep(10)
else:
    print(f"\n⚠ Timeout waiting for Gateway")
    raise Exception("Gateway timeout")


### ステップ 1.9: サンプルツールの Gateway への登録

サンプルツール Lambda（従業員データ）をデプロイし、Gateway ターゲットとして登録します。

In [ ]:
# Deploy tool Lambdas and register as Gateway targets
print("Deploying tool Lambda functions...")

# Import tool modules
sys.path.insert(0, str(Path.cwd()))
from src.tools import employee_data_tool

# Create IAM role for tool Lambdas using utils
TOOL_ROLE_ARN = utils.create_lambda_role(
    role_name=f"tool-lambda-role-{DEPLOYMENT_ID}",
    description='Role for tool Lambda functions'
)

# Deploy tool Lambda functions
tools_to_deploy = [
    ('employee_data_tool', employee_data_tool),
]

deployed_tools = []

for tool_name, tool_module in tools_to_deploy:
    print(f"  Deploying {tool_name}...")
    
    function_name = f"{tool_name.replace('_', '-')}-{DEPLOYMENT_ID}"
    tool_code_path = Path(tool_module.__file__)
    
    lambda_arn = utils.deploy_lambda_function(
        function_name=function_name,
        role_arn=TOOL_ROLE_ARN,
        lambda_code_path=str(tool_code_path),
        environment_vars={'TOOL_NAME': tool_name},
        description=f'{tool_name} function',
        region=REGION
    )
    
    tool_definition = getattr(tool_module, 'TOOL_DEFINITION', {
        "name": tool_name,
        "description": f"{tool_name} function"
    })
    
    deployed_tools.append({
        'tool_name': tool_name,
        'function_name': function_name,
        'lambda_arn': lambda_arn,
        'tool_definition': tool_definition
    })

print(f"✓ Deployed {len(deployed_tools)} tool Lambdas")

# Register tools as Gateway targets
print("\nRegistering tools as Gateway targets...")
created_targets = []

for tool in deployed_tools:
    print(f"  Registering {tool['tool_name']}...")
    
    try:
        response = gateway_client.create_gateway_target(
            gatewayIdentifier=GATEWAY_ID,
            name=f"{tool['tool_name'].replace('_', '-')}-target",
            targetConfiguration={
                "mcp": {
                    "lambda": {
                        "lambdaArn": tool["lambda_arn"],
                        "toolSchema": {"inlinePayload": [tool["tool_definition"]]}
                    }
                }
            },
            credentialProviderConfigurations=[{
                "credentialProviderType": "GATEWAY_IAM_ROLE"
            }]
        )
        
        target_id = response['targetId']
        print(f"    ✓ Target created: {target_id}")
        
        # Wait for target to be READY
        for attempt in range(18):
            status_response = gateway_client.get_gateway_target(
                gatewayIdentifier=GATEWAY_ID,
                targetId=target_id
            )
            status = status_response.get('status')
            
            if status == 'READY':
                print(f"    ✓ Target is READY")
                created_targets.append({
                    'tool_name': tool['tool_name'],
                    'target_id': target_id,
                    'lambda_arn': tool['lambda_arn']
                })
                break
            elif status == 'FAILED':
                print(f"    ✗ Target FAILED")
                break
            
            time.sleep(10)
            
    except Exception as e:
        print(f"    ✗ Failed to create target: {e}")

# Summary
print(f"\n✓ Deployed {len(deployed_tools)} tool Lambdas")
print(f"✓ Created {len(created_targets)} gateway targets")

if len(created_targets) < len(deployed_tools):
    print(f"⚠ Warning: Not all targets were created successfully")

# Store for cleanup
DEPLOYED_TOOL_FUNCTIONS = [t['function_name'] for t in deployed_tools]
CREATED_TARGET_IDS = [t['target_id'] for t in created_targets]


---

## パート 2: テスト

### ステップ 2.1: Bedrock Guardrails による PII 匿名化のテスト

従業員データツールを呼び出し、レスポンス内の PII が匿名化されていることを確認します。

#### 期待される結果：

従業員データツールは、さまざまな種類の PII を含む現実的な従業員情報を返します。Lambda インターセプターは以下を行います：

1. **レスポンスをインターセプト** ツールが実行された後
2. **Bedrock Guardrails を適用** 31 以上のエンティティタイプにわたって PII を検出
3. **検出された PII を匿名化** プレースホルダートークンに置換
4. **サニタイズされたレスポンスを返す** クライアントに

#### Bedrock Guardrails 匿名化フォーマット：

Bedrock Guardrails は検出された PII を以下の形式の匿名化されたプレースホルダーに置換します：

- **メール**: `john.doe@example.com` → `[EMAIL]`
- **電話番号**: `+1-555-123-4567` → `[PHONE]`
- **名前**: `John Doe` → `[NAME]`
- **住所**: `123 Main St, Springfield, IL 62701` → `[ADDRESS]`
- **SSN**: `123-45-6789` → `[US_SOCIAL_SECURITY_NUMBER]`
- **クレジットカード**: `4532-1234-5678-9010` → `[CREDIT_DEBIT_CARD_NUMBER]`
- **IP アドレス**: `192.168.1.1` → `[IP_ADDRESS]`
- **URL**: `https://example.com` → `[URL]`

#### 出力例：

**匿名化前（生のツールレスポンス）：**
```json
{
  "employee_id": "EMP-98765",
  "department": "Engineering",
  "contact_info": "alice.smith@company.com",
  "mailing_info": "456 Oak Avenue, Boston, MA 02101",
  "status": "Active"
}
```

**匿名化後（インターセプトされたレスポンス）：**
```json
{
  "employee_id": "EMP-98765",
  "department": "Engineering",
  "contact_info": "[EMAIL]",
  "mailing_info": "[ADDRESS]",
  "status": "Active"
}
```

`employee_id`、`department`、`status` などの非機密データは変更されず、すべての PII（メール、住所）は匿名化されたプレースホルダーに置換されていることに注意してください。

In [ ]:
# Test the PII masking by invoking the tool
import requests

print("Testing PII masking interceptor...")
print(f"Gateway URL: {GATEWAY_URL}")

# Get OAuth token
token_data = utils.get_token(
    user_pool_id=USER_POOL_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope_string="gateway/tools",
    REGION=REGION
)

if 'error' in token_data:
    print(f"✗ Token request failed: {token_data['error']}")
else:
    token = token_data['access_token']
    print(f"✓ Token obtained")

### ステップ 2.2: 従業員データツールのテスト

従業員データツールを呼び出して、フィールド名が明示的に機密性を示していない場合でも、Bedrock Guardrails がどのように連絡先情報と財務データを含む包括的な PII を匿名化するかを確認します。

従業員データツールは以下を返します：
- **連絡先情報**: メールと物理的住所
- **財務情報**: 銀行口座番号、ルーティング番号、クレジットカード詳細、CVV、有効期限、PIN、税務 ID
- **非機密データ**: 従業員 ID、部門、ステータス、口座残高、クレジットスコア

**匿名化前：**
```json
{
  "employee_id": "EMP-98765",
  "department": "Engineering",
  "contact_info": "alice.smith@company.com",
  "mailing_info": "456 Oak Avenue, Boston, MA 02101",
  "status": "Active",
  "financial_info": {
    "bank_account": "123456789",
    "routing_number": "987654321",
    "credit_card": "4532-1234-5678-9010",
    "cvv": "123",
    "card_expiry": "12/28",
    "pin": "1234",
    "tax_id": "987-65-4321",
    "account_balance": 25000.50,
    "credit_score": 750
  }
}
```

**匿名化後：**
```json
{
  "employee_id": "EMP-98765",
  "department": "Engineering",
  "contact_info": "[EMAIL]",
  "mailing_info": "[ADDRESS]",
  "status": "Active",
  "financial_info": {
    "bank_account": "[US_BANK_ACCOUNT_NUMBER]",
    "routing_number": "[US_BANK_ROUTING_NUMBER]",
    "credit_card": "[CREDIT_DEBIT_CARD_NUMBER]",
    "cvv": "[CREDIT_DEBIT_CARD_CVV]",
    "card_expiry": "[CREDIT_DEBIT_CARD_EXPIRY]",
    "pin": "[PIN]",
    "tax_id": "[US_INDIVIDUAL_TAX_IDENTIFICATION_NUMBER]",
    "account_balance": 25000.50,
    "credit_score": 750
  }
}
```

**主な観察点：**
- **コンテンツベースの検出**: `contact_info` や `mailing_info` などのフィールド名は「メール」や「住所」と明示的に言っていませんが、Bedrock Guardrails はパターン認識に基づいてコンテンツを検出し匿名化します
- **包括的な財務 PII 保護**: すべての機密財務データ（銀行口座、クレジットカード、税務 ID）が自動的に検出され匿名化されます
- **選択的匿名化**: 口座残高やクレジットスコアなどの非機密財務データは変更されません
- **31 以上の PII タイプ**: Bedrock Guardrails は明示的なフィールド名マッチングを必要とせずに、幅広い PII タイプを保護します

In [ ]:
# Test the employee data tool
print("\n" + "="*60)
print("Testing Employee Data Tool with PII Anonymization")
print("="*60)

# Reuse the token from previous step
if 'token' in locals():
    print(f"✓ Using existing token")
    
    # Call the employee data tool
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 2,
        "params": {
            "name": "employee-data-tool-target___employee_data_tool",
            "arguments": {"employee_id": "EMP-98765"}
        }
    }
    
    response = requests.post(
        GATEWAY_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=mcp_request
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"\n✓ Employee tool invoked successfully")
        print(f"\nResponse (PII in contact_info and mailing_info should be anonymized):")
        print(json.dumps(result, indent=2))
        
        # Highlight the anonymization
        print(f"\n📝 Notice:")
        print(f"  - 'employee_id', 'department', and 'status' remain unchanged (non-sensitive)")
        print(f"  - 'contact_info' email is replaced with [EMAIL] placeholder")
        print(f"  - 'mailing_info' address is replaced with [ADDRESS] placeholder")
        print(f"  - All financial PII is anonymized:")
        print(f"    • Bank account → [US_BANK_ACCOUNT_NUMBER]")
        print(f"    • Routing number → [US_BANK_ROUTING_NUMBER]")
        print(f"    • Credit card → [CREDIT_DEBIT_CARD_NUMBER]")
        print(f"    • CVV → [CREDIT_DEBIT_CARD_CVV]")
        print(f"    • Card expiry → [CREDIT_DEBIT_CARD_EXPIRY]")
        print(f"    • PIN → [PIN]")
        print(f"    • Tax ID → [US_INDIVIDUAL_TAX_IDENTIFICATION_NUMBER]")
        print(f"  - Non-sensitive financial data preserved (balances, credit scores, currency)")
        print(f"\n  ⭐ Key Point: Bedrock Guardrails uses content-based detection, not field names.")
        print(f"  Field names like 'mailing_info' don't explicitly say 'address', but the content")
        print(f"  is still detected and anonymized. This works across 31+ PII types automatically!")
    else:
        print(f"✗ Request failed: {response.status_code}")
        print(f"Response: {response.text}")
else:
    print("✗ No token available. Please run Step 2.1 first.")

---

# パート 3: クリーンアップ - すべてのリソースを削除

⚠️ **警告: これによりパート 1 で作成されたすべてのリソースが削除されます！**

すべてをクリーンアップしたい場合にのみこのセクションを実行してください。

### ステップ 3.1: 作成されたリソースの削除

In [ ]:
# Cleanup - Delete all created resources using utils
print("Starting cleanup...")

# 1. Delete gateway targets
if 'CREATED_TARGET_IDS' in globals() and 'GATEWAY_ID' in globals():
    utils.delete_gateway_targets(gateway_client, GATEWAY_ID, CREATED_TARGET_IDS)
    # Wait for target deletions to complete before deleting gateway
    time.sleep(5)

# 2. Delete gateway
if 'GATEWAY_ID' in globals():
    utils.delete_gateway(gateway_client, GATEWAY_ID)
    print("✓ Deleted gateway")

# 3. Delete Lambda functions (tools + interceptor)
lambda_functions_to_delete = []
if 'DEPLOYED_TOOL_FUNCTIONS' in globals():
    lambda_functions_to_delete.extend(DEPLOYED_TOOL_FUNCTIONS)
if 'LAMBDA_FUNCTION_NAME' in globals():
    lambda_functions_to_delete.append(LAMBDA_FUNCTION_NAME)

if lambda_functions_to_delete:
    utils.delete_lambda_functions(lambda_functions_to_delete, REGION)

# 4. Delete IAM roles
if 'LAMBDA_ROLE_NAME' in globals():
    utils.delete_iam_role(LAMBDA_ROLE_NAME)
if 'DEPLOYMENT_ID' in globals():
    utils.delete_iam_role(f"tool-lambda-role-{DEPLOYMENT_ID}")
    utils.delete_iam_role(f"agentcore-{GATEWAY_NAME}-role")

# 5. Delete Cognito user pool
if 'USER_POOL_ID' in globals():
    utils.delete_cognito_user_pool(USER_POOL_ID, REGION)

# 6. Delete Bedrock Guardrail
if 'GUARDRAIL_ID' in globals():
    try:
        print("\nDeleting Bedrock Guardrail...")
        bedrock_client.delete_guardrail(guardrailIdentifier=GUARDRAIL_ID)
        print(f"✓ Deleted guardrail: {GUARDRAIL_ID}")
    except Exception as e:
        print(f"⚠ Failed to delete guardrail: {e}")

print("\n✓ Cleanup complete!")

---

# サマリー

このノートブックでは Lambda インターセプターを使用した PII マスキングを示しました：

1. ✅ **セットアップ** - Lambda インターセプター、IAM ロール、Cognito、Gateway を作成
2. ✅ **テスト** - Gateway レスポンスを通じて PII マスキングを検証
3. ✅ **クリーンアップ** - すべてのリソースを削除

## 実演した内容

- ツールレスポンス内の機密データをマスクする **Lambda レスポンスインターセプター**
- 正規表現パターンを使用した **PII 検出とマスキング**
- カスタムインターセプターとの **Gateway 統合**
- 完全な**リソースライフサイクル**管理

## 次のステップ

- ユースケースに合わせてマスキングパターンをカスタマイズ
- より洗練された PII 検出を追加（例：AWS Comprehend）
- コンプライアンスログとの統合
- デバッグ用に CloudWatch ログを監視